In [ ]:
# Requires the datasets to be loaded

# correct datasets:

train_set_ood = pd.read_csv('ood_train_set_2.csv')
test_set_ood = pd.read_csv('ood_test_set_2.csv')

train_set_wd = pd.read_csv('wd_train_set_org.csv').rename(columns={'aph_443':'org_aph_443','aNAP_443':'org_anap_443','aCDOM_443':'org_acdom_443'})
test_set_wd = pd.read_csv('wd_test_set_org.csv').rename(columns={'aph_443':'org_aph_443','aNAP_443':'org_anap_443','aCDOM_443':'org_acdom_443'})

train_set_random = pd.read_csv('random_df_train_org.csv').rename(columns={'aph_443':'org_aph_443','aNAP_443':'org_anap_443','aCDOM_443':'org_acdom_443'})
test_set_random = pd.read_csv('random_df_test_org.csv').rename(columns={'aph_443':'org_aph_443','aNAP_443':'org_anap_443','aCDOM_443':'org_acdom_443'})

In [ ]:
# not used anymore, can skip

import seaborn as sns
import matplotlib.pyplot as plt

def plot_cdf_combined(train_set_random, test_set_random, train_set, test_set, train_set_oos, test_set_oos, figsize=(18, 12)):
    fig, axes = plt.subplots(3, 3, figsize=figsize, constrained_layout=True)

    # Define the column labels for the x-axis
    x_labels = [r'a$_{ph}$443 [m$^{-1}$]', r'a$_{nap}$443 [m$^{-1}$]', r'a$_{cdom}$443 [m$^{-1}$]']
    
    # Plot for random split set (first row)
    columns = ['org_aph_443', 'org_anap_443', 'org_acdom_443']
    for idx, col in enumerate(columns):
        sns.ecdfplot(train_set_random[col].clip(lower=0.000001), label="Train" if idx == 0 else "_nolegend_", ax=axes[0, idx])
        sns.ecdfplot(test_set_random[col].clip(lower=0.000001), label="Test" if idx == 0 else "_nolegend_", ax=axes[0, idx])
        axes[0, idx].set_xscale('log')
        axes[0, idx].set_title("Random split datasets" if idx == 1 else "")
        axes[0, idx].set_xlabel('')
        axes[0, idx].set_ylabel('Proportion' if idx == 0 else '')
        axes[0, idx].set_ylabel('')
        axes[0,idx].grid(True, ls='--',alpha=0.6)  # Enable gridlines

    # Plot for train and test set (WD) (second row)
    columns = ['org_aph_443', 'org_anap_443', 'org_acdom_443']
    for idx, col in enumerate(columns):
        sns.ecdfplot(train_set[col], label="Train" if idx == 0 else "_nolegend_", ax=axes[1, idx])
        sns.ecdfplot(test_set[col], label="Test" if idx == 0 else "_nolegend_", ax=axes[1, idx])
        axes[1, idx].set_xscale('log')
        axes[1, idx].set_title("Within-distribution (WD) datasets" if idx == 1 else "")
        axes[1, idx].set_xlabel('')
        axes[1, idx].set_ylabel('')
        axes[1,idx].grid(True, ls='--',alpha=0.6)  # Enable gridlines

    # Plot for out-of-sample (OOS) set (third row)
    columns = ['org_aph_443', 'org_anap_443', 'org_acdom_443']
    for idx, col in enumerate(columns):
        sns.ecdfplot(train_set_oos[col], label="Train OOS" if idx == 0 else "_nolegend_", ax=axes[2, idx])
        sns.ecdfplot(test_set_oos[col], label="Test OOS" if idx == 0 else "_nolegend_", ax=axes[2, idx])
        axes[2, idx].set_xscale('log')
        axes[2, idx].set_title("Out-of-distribution (OOD) datasets" if idx == 1 else "")
        axes[2, idx].set_xlabel(f'{x_labels[idx]}', fontsize=14)
        axes[2, idx].set_ylabel('')
        axes[2,idx].grid(True, ls='--',alpha=0.6)  # Enable gridlines

    for row in axes:
        for ax in row:
            ax.set_xlim(10**-3, 10**1)

    # Set super ylabel for the entire figure
    fig.supylabel('Proportion', x=0.01, fontsize=14, fontweight='bold')
    fig.supxlabel('IOP', fontsize=14, fontweight='bold')

    # Create a common legend for the entire figure
    handles, labels = axes[0,0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(1.015, 0.5), ncol=1)

    plt.tight_layout()
    #plt.savefig(r'C:\SwitchDrive\Data\Update_4\plots\all_data_split.png',dpi=200,bbox_inches='tight')
    plt.show()

plot_cdf_combined(train_set_random, test_set_random, train_set_wd, test_set_wd, train_set_ood, test_set_ood, figsize=(18, 12))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_log_histograms_only(train_set_random, test_set_random, train_set, test_set, train_set_oos, test_set_oos, columns, figsize=(14, 8)):
    fig, axes = plt.subplots(3, len(columns), figsize=figsize, constrained_layout=True)

    # Define the column labels for the x-axis
    x_labels = [r'a$_{ph}$443 [m$^{-1}$]', r'a$_{nap}$443 [m$^{-1}$]', r'a$_{cdom}$443 [m$^{-1}$]']

    # Update titles for specific subplots
    custom_titles = {
        (0, 1): 'Random split',
        (1, 1): 'Within-distribution split',
        (2, 1): 'Out-of-distribution split'
    }

    datasets = [
        (train_set_random, test_set_random, "Random split datasets"),
        (train_set, test_set, "Within-distribution (WD) datasets"),
        (train_set_oos, test_set_oos, "Out-of-distribution (OOS) datasets")
    ]

    x_limits = [(0.001, 10), (0.001, 10), (0.001, 10)]
    y_limits = [(0, 300), (0, 300), (0, 300)]

    for row, (train_data, test_data, title) in enumerate(datasets):
        for idx, col in enumerate(columns):
            # Define the bins with a logarithmic scale
            bins = np.logspace(np.log10(0.001), np.log10(10), 20)
            sns.histplot(train_data[col], bins=bins, kde=False, ax=axes[row, idx], label="Train", color="black", alpha=0.7, element="step", fill=True)
            sns.histplot(test_data[col], bins=bins, kde=False, ax=axes[row, idx], label="Test", color="orange", alpha=0.5, element="step", fill=True)
            axes[row, idx].set_xscale('log')
            axes[row, idx].set_xlim(*x_limits[idx])
            axes[row, idx].set_ylim(*y_limits[idx])
            axes[row,idx].set_ylabel("")
            axes[row, idx].set_xlabel(x_labels[idx] if row == 2 else '', fontsize=14)
            if (row, idx) in custom_titles:
                axes[row, idx].set_title(custom_titles[(row, idx)], fontweight='bold')
            else:
                axes[row, idx].set_title('')
            axes[row, idx].grid(True, linestyle='--', alpha=0.6)

    # Set super ylabel for the entire figure
    fig.supylabel('Frequency', x=-0.03, y=0.55, fontsize=12, fontweight='bold')
    fig.supxlabel('IOP', fontsize=12, fontweight='bold')

    # Adjust the subplots and create a common legend
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(1.05, 0.57), ncol=1, frameon=True, fontsize=14)

    #plt.tight_layout()
    plt.savefig(r'C:\SwitchDrive\Data\Plots\hist_all_data_split.png', dpi=200, bbox_inches='tight')
    plt.show()

# Usage example, assuming the same datasets and columns are defined
columns = ['org_aph_443', 'org_anap_443', 'org_acdom_443']
plot_log_histograms_only(train_set_random, test_set_random, train_set_wd, test_set_wd, train_set_ood, test_set_ood, columns)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

# Example: Assuming 'robust_test_df' and 'robust_train_df' are already defined
# Concatenating the dataframes
data = pd.concat([df1, df2])[['aCDOM_443', 'aCDOM_675',
                                                    'aNAP_443', 'aNAP_675',
                                                    'aph_443', 'aph_675']]

# Defining the thresholds for each column
thresholds = {
    'aCDOM_443': 10**-3,
    'aCDOM_675': 10**-5,
    'aNAP_443': 10**-3,
    'aNAP_675': 10**-5,
    'aph_443': 10**-3,
    'aph_675': 10**-5
}

colors = {
    'aCDOM_443': 'darkgoldenrod',
    'aCDOM_675': 'darkgoldenrod',
    'aNAP_443': 'chocolate',
    'aNAP_675': 'chocolate',
    'aph_443': 'darkgreen',
    'aph_675': 'darkgreen'
}
offsets = {
    'aCDOM_443': (1.3, 110),
    'aCDOM_675': (0.08, 110),
    'aNAP_443': (0.9, 110),
    'aNAP_675': (0.19, 110),
    'aph_443': (0.95, 110),
    'aph_675': (0.7, 110)
}


# Custom x-labels
x_labels = ['a$_{CDOM}$443 [m$^{-1}$]', 'a$_{CDOM}$675 [m$^{-1}$]', 'a$_{NAP}$443 [m$^{-1}$]', 'a$_{NAP}$675 [m$^{-1}$]', 'a$_{ph}$443 [m$^{-1}$]', 'a$_{ph}$675 [m$^{-1}$]']

# Filtering data based on the thresholds
for col, threshold in thresholds.items():
    data = data[data[col] > threshold]

# Setting up the plot with 2 rows and 3 columns
fig, axes = plt.subplots(3, 2, figsize=(12, 6))

# Flattening the axes array for easier iteration
axes = axes.ravel()

# Plotting each column in its respective subplot
for i, col in enumerate(data.columns):
    # Avoiding negative or zero values by filtering only positive entries
    filtered_data = data[col][data[col] > 0]
    mean_value = filtered_data.mean()

    # Plotting with log scale for both x and y axes
    axes[i].hist(filtered_data, bins=np.logspace(np.log10(filtered_data.min()), np.log10(filtered_data.max()), 50), color=colors[col])
    axes[i].set_xscale('log')
    axes[i].set_xlabel(x_labels[i])
    # Set different x-limits based on the column
    if '_675' in col:
        axes[i].set_xlim(10**-5, 10**1)
    else:
        axes[i].set_xlim(10**-3, 10**1)
    axes[i].set_ylim(0,200)
    #axes[i].axvline(mean_value, color='black', linestyle='dashed', linewidth=1)
    offset = offsets[col]
    #axes[i].annotate(f'$\mu$: {mean_value:.2}', xy=(mean_value+offset[0], offset[1]), xytext=(mean_value, 10),
    #                 textcoords='offset points', ha='center', va='bottom')
    axes[i].grid(True, ls='--',color='black',alpha=0.2)

# Adding a common y-axis label
fig.supylabel('Frequency',fontweight='bold')

# Adding wavelength column titles
plt.figtext(0.275, 1, '443 nm', ha='center', va='center', fontsize=12,fontweight='bold')
plt.figtext(0.775, 1, '675 nm', ha='center', va='center', fontsize=12,fontweight='bold')

# Adjusting layout to prevent overlap
plt.tight_layout()
plt.savefig(r'C:\SwitchDrive\Data\Plots\iop_hist.png', dpi=200, bbox_inches='tight')
plt.show()
